In [42]:
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, LabelSet, CustomJS, Slider
import pandas as pd
import os

In [43]:
output_notebook()

Loading BokehJS ...

In [44]:
pathN = os.path.dirname(os.path.realpath('__file__'))
filename = 'Years in school, women 25 to 34 as percent of men.xlsx'
filename_X = 'indicator gapminder gdp_per_capita_ppp.xlsx'
xl = pd.ExcelFile(pathN + '/' + filename)
xl_X = pd.ExcelFile(pathN + '/' + filename_X)
data = xl.parse('Data')
data_X = xl_X.parse('Data')


In [45]:
rows, columns = data.shape
#print (rows, columns)
data_india = data.loc[data['Row Labels'] == 'India']
data_US = data.loc[data['Row Labels'] == 'United States']

hover = HoverTool(
        tooltips=[
            ('Year', '$x{int}'),
            ('Years in School (%)', '$y')
        ]
    )

p = figure(plot_width = 700, plot_height = 400, tools = [hover, 'pan', 'wheel_zoom', 'reset', 'resize','box_zoom','tap','previewsave','box_select'], title = 'Years in school, women 25 to 34 as percent of men')
p.circle(list(data_US.columns.values)[1:], list(data_US.iloc[0])[1:], size=10, line_color = 'navy', fill_color = 'orange', legend = 'US', alpha = 0.8, muted_color = 'orange', muted_alpha=0.2)
#print (data_india)
p.circle(list(data_india.columns.values)[1:], list(data_india.iloc[0])[1:], size=10, line_color = 'navy', legend = 'India', alpha = 0.8, muted_alpha=0.2)
p.legend.location = 'bottom_right'
p.legend.click_policy = 'mute'
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Mean Years in school, women 25 to 34, % of men'
show(p)

The plot above shows US and India's glyphs from 1970 to 2009. The legend can be clicked to mute/unmute the lines, and there are hover tooltips to display x and y coordinates.

## Adding Interactivity

In the next plot, an interactive slider is added to show time series data, and new GDP data on the x-axis is added. 

In [58]:
names = []
for i in range(0, rows):
    names.append(data.iloc[i,0])
    
gdp = []
final_names = []
begin_year = data.columns.values[1]
current_year = begin_year
end_year = data.columns.values[-1]
for i in range(0, rows):
    val = list(data_X.loc[data_X['GDP per capita'] == names[i], begin_year].values)
    if len(val) == 1:
        gdp.append(val[0])
        final_names.append(names[i])
        
Y_inSchool = []
for i in range(0, len(final_names)):
    Y_inSchool.append(list(data.loc[data['Row Labels'] == final_names[i], begin_year].values)[0])

yData = []
for i in range(0,len(final_names)):
    yData.append(data.loc[data['Row Labels'] == final_names[i]].values[0].tolist())
    
xData = []
for i in range(0,len(final_names)):
    xData.append(data_X.loc[data_X['GDP per capita'] == final_names[i]].values[0].tolist())

colors = []
for i in range(0, len(final_names)):
    colors.append('#%02x%02x%02x' % (int(255*(gdp[i]-min(gdp))/(max(gdp)-min(gdp))),int(255*(Y_inSchool[i]-min(Y_inSchool))/(max(Y_inSchool)-min(Y_inSchool))),200))
    
source = ColumnDataSource(data=dict(
    xAxis=gdp,
    yAxis=Y_inSchool,
    names=final_names,
    colors = colors))

source2 = ColumnDataSource(data=dict(
    xData=xData,
    yData=yData,
    names=final_names))

hover = HoverTool(
        tooltips=[
            ('Country', '@names'),
            ('Income/person', '$x{1.11}'),
            ('Years in School(%)', '$y')
        ]
    )

callback = CustomJS(args=dict(source=source,source2=source2), code="""
    var data = source.data
    var begin_year = 1970
    var gdp = data['xAxis']
    var Y_inSchool = data['yAxis']
    var current_year = cb_obj.value
    for (i=0; i < source2.data.names.length; i++){
        gdp[i] = source2.data.xData[i][171+current_year-begin_year]
        Y_inSchool[i] = source2.data.yData[i][1+current_year-begin_year]
    }
    source.trigger('change');
    """)

newP = figure(plot_width = 900, plot_height = 500, x_axis_type='log', tools = [hover, 'pan', 'wheel_zoom', 'reset', 'resize','box_zoom','tap','previewsave','box_select', 'lasso_select'], title = 'Years in school, women 25 to 34 as percent of men')
newP.scatter(x='xAxis', y='yAxis', source=source, size= 8, fill_color = 'colors', fill_alpha = 0.6, line_color = 'blue')
newP.xaxis.axis_label = 'Income per person (GDP per capita)'
newP.yaxis.axis_label = 'Mean Years in school, women 25 to 34, % of men'

slider = Slider(start=begin_year, end=end_year, value=current_year, step=1, title="Year", callback=callback)

layout = column(newP,slider)

show(layout)

The above graph contains all countries (countries with no GDP data were removed). The slide on the bottom indicates the year and goes from 1970 to 2009. The size of glyphs is constant and the color is assigned at the beginning using a mild, linear color scale.
The x-axis is on a log scale, y-axis being linear. The controls on the right can be used to resize the plot, select glyphs, zoom, save, and reset. The hover tooltips display the country and X and Y information.